In [21]:
# import gdown ; 
import pandas as pd ; 
import numpy as np;
import math
import os
import glob
# train_log_url = "https://drive.google.com/file/d/1VebJgr8LlRWeCsAnYA2OK6VzdtsB9L83/view?usp=drive_link"
# gdown.download(url=train_log_url, quiet=False, fuzzy=True)

# train_score_url = "https://drive.google.com/file/d/1VQ05dODI0aKyvu2RB0-WU2hzk4_bqJyw/view?usp=sharing"
# gdown.download(url=train_score_url, quiet=False, fuzzy=True)

In [30]:
##function to apply on each row for train_logs
def pause_time_to_string(value):
    ## If the pause is bigger than 1000(1sec), value/1000 that many pause to column
    if math.isnan(value)or value <= 1000 :
        return ""
    else:
        sec = round(value/ 1000)
        return " ".join(['pause' for i in range(sec)])

In [31]:
def combine_columns(df):
    if df['pause_str'] != '':
        return df['down_event'] + " " + df['pause_str'] 
    else:
        return df['down_event']

In [32]:
##function to generate and return a string of hmm sequence array
def genHMMInput(df):
    df = df.copy()
    ##create a new coulmn that contains pause time
    df["next_down_time"] = df["down_time"].shift(-1)
    df["pause_time"] = df["next_down_time"] - df["down_time"]
    ####create new column pause_str.
    df["pause_str"] = df['pause_time'].apply(pause_time_to_string)
    df['hmm_input'] = df.apply(combine_columns, axis=1)
    mask = df['id'] != df['id'].shift(-1)
    mask.iloc[-1] = False
    df.loc[mask, 'hmm_input'] += " K"
    hmm_input = " ".join(df["hmm_input"].tolist())
    return hmm_input

In [5]:
# # df2 = pd.read_csv("train_logs.csv")
# df = df2.loc[0:4].copy()
# df["next_down_time"] = df["down_time"].shift(-1)
# df["pause_time"] = df["next_down_time"] - df["down_time"]
# ####create new column pause_str.
# df["pause_str"] = df['pause_time'].apply(pause_time_to_string)
# df['hmm_input'] = df.apply(combine_columns, axis=1)
# mask = df['id'] != df['id'].shift(-1)
# mask.iloc[-1] = False
# df.loc[mask, 'hmm_input'] += " K"
# hmm_input = " ".join(df["hmm_input"].tolist())
# hmm_input

In [6]:
##function that call the above and write them into files
def generateFileBaseOnScore(logs, score):
    train_dfs = []
    test_dfs = []
    
    logs_df = pd.read_csv(logs)
    score_df  = pd.read_csv(score)

    if not os.path.exists("train"):
            os.makedirs("train")
    if not os.path.exists("test"):
            os.makedirs("test")
    ##generate inner join dataframe
    log_join_score = pd.merge(logs_df, score_df, on="id", how='inner')
    for s in score_df["score"].unique():
        ##join log and score dataframe base on id
        cur_score_df = log_join_score.loc[log_join_score["score"] == s]
        unique_doc_id = cur_score_df["id"].unique()
        num_unqiue = len(unique_doc_id)
        ##set up randomness and raito for train/test split
        np.random.shuffle(unique_doc_id)
        split_ratio = 0.8
        split_index = int(len(unique_doc_id) * split_ratio)

        ##set the ids that are in train set and test set
        train_id = unique_doc_id[:split_index]
        test_id = unique_doc_id[split_index:]

        len_train = len(train_id)
        len_test = len(test_id)
        print(f"Total of {num_unqiue} for score {s}, train will have {len_train} and test will have {len_test}")

        ##create df for train and test
        train_df = log_join_score[log_join_score['id'].isin(train_id)]
        test_df = log_join_score[log_join_score['id'].isin(test_id)]

        ##add them to the list
        train_dfs.append(train_df)
        test_dfs.append(test_df)

        for df in train_dfs:
            s = str(s)
            hmm_input = genHMMInput(df)
            with open( "train/" +s +".txt", 'w') as file:
                file.write(hmm_input)
        for df in test_dfs:
            hmm_input = genHMMInput(df)
            with open( "test/" +s +".txt", 'w') as file:
                file.write(hmm_input)
    

generateFileBaseOnScore("train_logs.csv", "train_scores.csv")

Total of 486 for score 3.5, train will have 388 and test will have 98
Total of 37 for score 6.0, train will have 29 and test will have 8
Total of 92 for score 2.0, train will have 73 and test will have 19
Total of 501 for score 4.0, train will have 400 and test will have 101
Total of 402 for score 4.5, train will have 321 and test will have 81
Total of 201 for score 2.5, train will have 160 and test will have 41
Total of 179 for score 5.0, train will have 143 and test will have 36
Total of 336 for score 3.0, train will have 268 and test will have 68
Total of 69 for score 1.5, train will have 55 and test will have 14
Total of 128 for score 5.5, train will have 102 and test will have 26
Total of 35 for score 1.0, train will have 28 and test will have 7
Total of 5 for score 0.5, train will have 4 and test will have 1


In [40]:
def generateFileBaseOnEssay(source_folder):
    train_dfs = []
    test_dfs = []
    
    folder_path = "essay_hmm"
    if not os.path.exists(folder_path):
            os.makedirs(folder_path)
    score = [f for f in os.listdir(source_folder) if os.path.isdir(os.path.join(source_folder, f))]
    print(score)
    for folder in score:
        print(folder)
        csv_files = glob.glob(os.path.join(source_folder, folder, '*.csv'))
        for csv_file in csv_files:
            parts = csv_file.split('/')[-1].split("_")
            if len(parts) == 2 and parts[1].endswith('.csv'):
                essay_id =  parts[1][:-4]
            print(f"Reading essay {essay_id}")
            essay_df = pd.read_csv(csv_file)
            hmm_input = genHMMInput(essay_df)
            dest_folder = folder_path + "/" + folder
            if not os.path.exists(dest_folder):
                os.makedirs(dest_folder)
            with open(dest_folder +"/"+ essay_id + ".txt", "w") as file:
                file.write(hmm_input)        

generateFileBaseOnEssay("essay_csv")

['2.5', '6.0', '4.5', '1.0', '5.0', '3.0', '5.5', '4.0', '0.5', '3.5', '2.0', '1.5']
2.5
Reading essay e596b9ad
Reading essay 518d1052
Reading essay d5af6725
Reading essay 14198179
Reading essay 2e0fcc07
Reading essay c7aa7765
Reading essay e0ab02ae
Reading essay 6500aba6
Reading essay cd64ed13
Reading essay ea71cc1a
Reading essay 0144e4d5
Reading essay 89e03572
Reading essay 3814e70a
Reading essay 66ae4f3e
Reading essay cdc73008
Reading essay 9bb707cb
Reading essay 6e1048f2
Reading essay f7551231
Reading essay 54e4b9ea
Reading essay b816ee8e
Reading essay 473d6f24
Reading essay bd487e77
Reading essay 84957285
Reading essay 68a3f911
Reading essay 5e25026d
Reading essay b11e1fdc
Reading essay fa1a67ba
Reading essay 052cf3a6
Reading essay 8ee8c46c
Reading essay 9dad44fc
Reading essay ca8efe6c
Reading essay 6a46d751
Reading essay 35a8b483
Reading essay 3e0aa9dd
Reading essay fb6c9c09
Reading essay ea43e32d
Reading essay b66bdcfe
Reading essay c647d68a
Reading essay 8389a706
Reading essay 

In [16]:
# import pandas as pd

# #logs_df = pd.read_csv("train_logs.csv")
# score_df = pd.read_csv("train_scores.csv")
# log_join_score = pd.merge(logs_df, score_df, on="id", how='inner')
# grouped_score = log_join_score.groupby('score')

# for score, groupd_score_df in grouped_score:
#     folder_name = "eassy_csv"
#     if not os.path.exists(folder_name):
#             os.makedirs(folder_name)
#     essay_group = groupd_score_df.groupby('id')
#     for essay_id, essay_df in essay_group:
#         score = str(score)
#         if not os.path.exists(folder_name + "/" + score):
#             os.makedirs(folder_name + "/" + score)
#         file_name = f"essay_{essay_id}.csv"
#         file_path = folder_name + "/" + score + "/" + file_name
#         essay_df.to_csv(file_path, index=False)
#         print(f"essay_id {essay_id} written to {file_name}")


essay_id 1ebb9b74 written to essay_1ebb9b74.csv
essay_id 315bdafd written to essay_315bdafd.csv
essay_id 3bda31e6 written to essay_3bda31e6.csv
essay_id 40b28508 written to essay_40b28508.csv
essay_id c3663a2d written to essay_c3663a2d.csv
essay_id 044c5c54 written to essay_044c5c54.csv
essay_id 08e3c7ae written to essay_08e3c7ae.csv
essay_id 213d79d9 written to essay_213d79d9.csv
essay_id 2717fdef written to essay_2717fdef.csv
essay_id 32a001ec written to essay_32a001ec.csv
essay_id 3a266257 written to essay_3a266257.csv
essay_id 4f423d3a written to essay_4f423d3a.csv
essay_id 52bec922 written to essay_52bec922.csv
essay_id 58e307a8 written to essay_58e307a8.csv
essay_id 5a3f0d07 written to essay_5a3f0d07.csv
essay_id 5c88a769 written to essay_5c88a769.csv
essay_id 5e048ae3 written to essay_5e048ae3.csv
essay_id 65c2a095 written to essay_65c2a095.csv
essay_id 68df1430 written to essay_68df1430.csv
essay_id 69916fc0 written to essay_69916fc0.csv
essay_id 719ed6e1 written to essay_719ed

In [7]:
# ##write only 3.5 to a csv file
# logs_df = pd.read_csv("train_logs.csv")
# score_df = pd.read_csv("train_scores.csv")
# log_join_score = pd.merge(logs_df, score_df, on="id", how='inner')
# logs_df[log_join_score['score']==3.5].to_csv("35.csv")

In [8]:
# df = pd.read_csv("35.csv")
# df["next_down_time"] = df["down_time"].shift(-1)
# df["pause_time"] = df["next_down_time"] - df["down_time"]
# ####create new column pause_str.
# df["pause_str"] = df['pause_time'].apply(pause_time_to_string)
# df['hmm_input'] = df.apply(combine_columns, axis=1)
# mask = df['id'] != df['id'].shift(-1)
# mask.iloc[-1] = False
# df.loc[mask, 'hmm_input'] += " K"
# hmm_input = " ".join(df["hmm_input"].tolist())

In [9]:
# df[df['hmm_input'].str.contains('Å')]

In [10]:
# column_name = "pause_time"
# # plt.scatter(train_logs.index, train_logs[column_name])

# # # Add labels and title
# # plt.xlabel('Index')
# # plt.ylabel(column_name)
# # plt.title(f'Scatter Plot of {column_name} vs Index')

# # # Show the plot
# # # plt.show()

# percentiles = [25,50,75,80,85] + list(range(90,101))
# # percentiles = list(range(90,101))

# # Use the percentile function to calculate the specified percentiles
# percentile_values = np.percentile(train_logs[column_name].dropna(), percentiles)
# for p, value in zip(percentiles, percentile_values):
#     print(f'{p}th percentile: {value}')

In [11]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import numpy as np

# # Assuming you already have a DataFrame named 'df'
# # Replace 'column_name' with the actual column you want to plot
# column_name = 'action_time'
# # train_logs["action_time"] = train_logs["action_time"].astype(int)
# mask = (train_logs["action_time"] <=  10000)
# df = train_logs[mask]
# # Create a scatter plot
# plt.scatter(df.index, df[column_name])

# # Add labels and title
# plt.xlabel('Index')
# plt.ylabel(column_name)
# plt.title(f'Scatter Plot of {column_name} vs Index')

# # Show the plot
# plt.show()

# percentiles = list(range(90,101))

# # Use the percentile function to calculate the specified percentiles
# percentile_values = np.percentile(train_logs[column_name].dropna(), percentiles)
# for p, value in zip(percentiles, percentile_values):
#     print(f'{p}th percentile: {value}')

In [12]:
# # Assuming you already have a DataFrame named 'df'
# # Replace 'column_name' with the actual column you want to plot
# column_name = ''
# # train_logs["action_time"] = train_logs["action_time"].astype(int)
# mask = (train_logs["action_time"] <=  10000)
# df = train_logs[mask]
# # Create a scatter plot
# plt.scatter(df.index, df[column_name])

# # Add labels and title
# plt.xlabel('Index')
# plt.ylabel(column_name)
# plt.title(f'Scatter Plot of {column_name} vs Index')

# # Show the plot
# plt.show()

# percentiles = list(range(90,101))

# # Use the percentile function to calculate the specified percentiles
# percentile_values = np.percentile(train_logs[column_name].dropna(), percentiles)
# for p, value in zip(percentiles, percentile_values):
#     print(f'{p}th percentile: {value}')

In [13]:
# ##find the gap between down/up time
# print(max(train_logs["action_time"]), min(train_logs["action_time"]))

# for i, r in train_logs.iterrows():
#   cur_event = r["down_time"]